# Colab

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
False


In [0]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image

    100% |████████████████████████████████| 5.6MB 6.0MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())
import numpy as np
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

0.4.0
False


# Generating dataset

In [0]:
!pip3 install faker
!pip3 install babel

In [0]:
from date_utils import *

In [0]:
for i in range(5):
  x=load_date()
  print("Input (human-readable): "+str(x[0])+"\nTarget (machine-readable): "+str(x[1])+"\n")

Input (human-readable): 9 may 1998
Target (machine-readable): 1998-05-09

Input (human-readable): 10.09.70
Target (machine-readable): 1970-09-10

Input (human-readable): 4/28/90
Target (machine-readable): 1990-04-28

Input (human-readable): thursday january 26 1995
Target (machine-readable): 1995-01-26

Input (human-readable): monday march 7 1983
Target (machine-readable): 1983-03-07



In [0]:
m=10000 #Size of sample database
dataset,human_vocab,machine_vocab,inv_machine=load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 19382.69it/s]


In [0]:
dataset[:3]

In [0]:
#Converting inputs/targets from string to list of onehot-encoded chars
X,Y,X_oh,Y_oh=preprocess_data(dataset,human_vocab,machine_vocab,30,10)
#All inputs are converted to len=30 by adding '<pad>' element if necessary

In [0]:
print("Original\n"+str(dataset[5])+"\nConverted to ints\n"+
      str(X[5]) + " , " + str(Y[5])+"\nOne-hotted\n"+
      str(X_oh[5]) + " , " + str(Y_oh[5])+"\n")      

# Training

In [0]:
X_oh = X_oh.reshape([30,10000,37])
X_oh = torch.tensor(X_oh)
Y    = torch.tensor(Y)

In [0]:
print("X_oh shape: " + str(X_oh.shape)+"\nY shape:"+str(Y.shape))

X_oh shape: torch.Size([30, 10000, 37])
Y_oh shape:torch.Size([10, 10000, 11])


In [0]:
class Context(nn.Module):
  """
  Computes context vector from prev. hidden state [1x100x64] and activations [30x100x64]
  
  """
  def __init__(self,input_size,hidden_size): #hidden_size -> 10
    super(Context, self).__init__()   
    self.i2h = nn.Linear(input_size,hidden_size)
    self.tanh = nn.Tanh()
    self.h2o = nn.Linear(hidden_size,1)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(0) #dimension
    
  def forward(self, act, hid):
    """
    activation vector from pre-attention LSTM [30 x 100 x 64]
    previous hidden state of post-att. LSTM [1 x 100 x 64]
    
    """  
    output = torch.cat((act,hid.expand([30,-1,64])),2) #concatenating act & hid
    output = self.tanh(self.i2h(output))                       #[30 x 100 x hidden_size]
    output = self.relu(self.h2o(output))                       #[30 x 100 x 1]
    output = self.softmax(output)
    output = act * output
    output = torch.sum(output,0)
    output = output.view(1,-1,64)
    return output                                              #[1 x 100 x 64]

In [0]:
class post_LSTM(nn.Module):
  """
  Computes 1 output from the current context vector.
  h - LSTM hidden state
  c - LSTM memory unit
  
  """
  def __init__(self, ip_size, hid_size, op_size):
    super(post_LSTM, self).__init__()
    self.post_LSTM = nn.LSTM(input_size=ip_size,hidden_size=hid_size)
    self.o2o = nn.Linear(hid_size,op_size)
    self.softmax = nn.Softmax(1) #dimension
    
  def forward(self, context, h, c):
    #context -> [1 x 100 x ip_size]
    output, (h,c) = self.post_LSTM(context,(h,c))
    output = self.o2o(output)                       #[1 x 100 x hid_size]
    output = self.softmax(output)
    return output,(h,c)                             #[1 x 100 x op_size], h-> [1 x 100 x hid_size]

In [0]:
pre_BiLSTM = nn.LSTM(input_size=37, hidden_size=32,bidirectional=True)        #input  -> [30 x 100 x 37]   #output -> [30 x 100 x 64]
cxt_model = Context(128,10)
post_model = post_LSTM(64,64,11)
hs = torch.zeros([1,100,64]) #hidden state
mu = torch.zeros([1,100,64]) #memory unit
batch_output = torch.zeros([10,100,11])
loss_func = nn.NLLLoss()
optimizer1 = optim.Adam(params = pre_BiLSTM.parameters(),lr = 0.005,weight_decay = 0.01)
optimizer2 = optim.Adam(params = cxt_model.parameters(),lr = 0.005,weight_decay = 0.01)
optimizer3 = optim.Adam(params = post_model.parameters(),lr = 0.005,weight_decay = 0.01)


In [0]:
for i in (range(100)): #for each batch
  loss=0
  optimizer1.zero_grad()
  optimizer2.zero_grad()
  optimizer3.zero_grad()
  
  ii = i*100
  
  activations,(_,_) = pre_BiLSTM(X_oh[:,ii : ii + 100,:])
  
  for t in range(10):
    context = cxt_model(activations, hs)
    out,(hs,mu) = post_model(context,hs,mu)
    
    loss+=loss_func(out.view([100,11]),Y[ii:ii+100,t])
  loss.backward(retain_graph=True)
  optimizer1.step()
  optimizer2.step()
  optimizer3.step()
  
  if(i%10==0):
    print(loss)

# Model analysis

In [0]:
print(pre_BiLSTM)

LSTM(37, 32, bidirectional=True)


In [0]:
print(cxt_model)

Context(
  (i2h): Linear(in_features=128, out_features=10, bias=True)
  (tanh): Tanh()
  (h2o): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
  (softmax): Softmax()
)


In [0]:
print(post_model)

post_LSTM(
  (post_LSTM): LSTM(64, 64)
  (o2o): Linear(in_features=64, out_features=11, bias=True)
  (softmax): Softmax()
)


In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [0]:
count_parameters(pre_BiLSTM) + count_parameters(post_model) + count_parameters(cxt_model)

53472

# Testing


In [0]:
def predict(inp): #[]
  answer = ''
  with torch.no_grad():
    _,_,inp_oh,_=preprocess_data([inp,'0000000000'],human_vocab,machine_vocab,30,10)
    inp_oh = torch.tensor(inp_oh.view(30,1,37))
    
    activations,(_,_) = pre_BiLSTM(inp_oh)
  
    for t in range(10):
      context = cxt_model(activations, hs)
      out,(hs,mu) = post_model(context,hs,mu)
      out = out.view([1,-1])
      _,ans = torch.max(out,1)
      answer += inv_machine[int(ans)]

In [0]:
# softmax axis dimension in context
# retain_graph